In [217]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import random
from IPython.display import clear_output
from collections import deque
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [2]:
MNIST_train = pd.read_csv('MNIST_train.csv')
MNIST_test = pd.read_csv('MNIST_test.csv')

# Part A

We will use hierarchical k-means to build a dictionary of image patches.
For untouched images, construct a collection of 10 × 10 image patches.
You should extract these patches from the training images on an overlapping 4×4 grid, meaning that each training image produces 16 overlapping
patches (so you could have 960,000 training patches!). For each training
image, choose one of these patches uniformly and at random. Now subsample this dataset of 60,000 patches uniformly and at random to produce
a 6000 element dataset. Cluster this dataset to 50 centers. Now build 50
datasets, one per cluster center. Do this by taking each element of the
60,000 patch dataset, finding which of the cluster centers is closest to it,
and putting the patch in that center’s dataset. Now cluster each of these
datasets to 50 centers

In [32]:
def split_images_into_patches(image_id, image, dictionary):    
    image = (image.values).reshape(28, 28)
    image_dictionary = {}
    for it in range(16):
        #patch = []
        patch = deque()
        i_low = -1
        i_high = -1
        #count = 0
        
        if it >= 0 and it <= 3:
            i_low = 0
            i_high = 10
        elif it >= 4 and it <= 7:
            i_low = 4
            i_high = 14
        elif it >= 8 and it <= 11:
            i_low = 14
            i_high = 24
        else:
            i_low = 18
            i_high = 28
            
        for i in range(i_low, i_high):
            j_low = -1
            j_high = -1
            
            if it % 4 == 0:
                j_low = 18
                j_high = 28
            elif it % 3 == 0:
                j_low = 14
                j_high = 24
            elif it % 2 == 0:
                j_low = 4
                j_high = 14
            else:
                j_low = 0
                j_high = 10
            
            for j in range(j_low, j_high):
                patch.append(image[i][j])
                
        image_dictionary[it] = patch
    dictionary[image_id] = image_dictionary    

In [41]:
patch_dictionary = {}
for image_id in range(60000):
    image = MNIST_train.loc[:, MNIST_train.columns != 'labels'].iloc[image_id]
    image = (image.values).reshape(28, 28)
    image_dictionary = {}
    for it in range(16):
        patch = deque()
        i_low = -1
        i_high = -1
        
        if it >= 0 and it <= 3:
            i_low = 0
            i_high = 10
        elif it >= 4 and it <= 7:
            i_low = 4
            i_high = 14
        elif it >= 8 and it <= 11:
            i_low = 14
            i_high = 24
        else:
            i_low = 18
            i_high = 28
            
        for i in range(i_low, i_high):
            j_low = -1
            j_high = -1
            
            if it % 4 == 0:
                j_low = 18
                j_high = 28
            elif it % 3 == 0:
                j_low = 14
                j_high = 24
            elif it % 2 == 0:
                j_low = 4
                j_high = 14
            else:
                j_low = 0
                j_high = 10
            
            for j in range(j_low, j_high):
                patch.append(image[i][j])
                
        image_dictionary[it] = patch
    patch_dictionary[image_id] = image_dictionary 

In [42]:
col_names = []
for i in range(100):
    col_names.append("Pixel " + str(i+1))

In [43]:
random.seed(498)
patch_training_df = pd.DataFrame(columns = col_names)
for i in range(60000):
    choose_patch = random.randint(0,15)
    patch_training_df.loc[i] = patch_dictionary[i][choose_patch]

59999


In [107]:
random.seed(498)
patch_training_df_sample = patch_training_df.sample(6000)
indices = patch_training_df_sample.index

In [59]:
patch_cluster = KMeans(n_clusters=50)
patch_cluster_fit = patch_cluster.fit_predict(patch_training_df_sample)
patch_cluster_centers = patch_cluster.cluster_centers_

In [108]:
cluster_data_collections = []
for i in range(50):
    cluster_data_collections.append(pd.DataFrame(columns=col_names))

In [109]:
closest_center = patch_cluster.predict(patch_training_df)

In [110]:
for i in range(60000):
    cluster_data_collections[closest_center[i]] = cluster_data_collections[closest_center[i]].append(patch_training_df.loc[i])

In [111]:
cluster_center_collections = []
for i in range(50):
    internal_cluster = KMeans(n_clusters=50)
    internal_cluster_fit = internal_cluster.fit_predict(cluster_data_collections[i])
    cluster_center_collections.append(internal_cluster)

# Part B

You now have a dictionary of 2500 entries. For each query image, construct a set of 10 × 10 patches on an overlapping 4 × 4 grid. Now for
each of the centers, you should extract 9 patches. Assume the center is
at (x, y); obtain 9 patches by extracting a patch centered at (x − 1, y −
1),(x, y − 1),...,(x + 1, y + 1). This means each test image will have 144
associated patches. Now use your dictionary to find the closest center to
each patch, and construct a histogram of patches for each test image.

In [123]:
def split_image_into_smaller_patches(image_id, image, dictionary):
    image = (image.values).reshape(28, 28)
    image_dictionary = {}
    patch_list = []
    
    for it in range(16):
        patch_1 = []
        patch_2 = []
        patch_3 = []
        patch_4 = []
        patch_5 = []
        patch_6 = []
        patch_7 = []
        patch_8 = []
        patch_9 = []
        
        i_low = -1
        i_high = -1
        counter = 0
        
        if it >= 0 and it <= 3:
            i_low = 0
            i_high = 10
        elif it >= 4 and it <= 7:
            i_low = 4
            i_high = 14
        elif it >= 8 and it <= 11:
            i_low = 14
            i_high = 24
        else:
            i_low = 18
            i_high = 28
            
        for i in range(i_low, i_high):
            j_low = -1
            j_high = -1
            
            if it % 4 == 0:
                j_low = 18
                j_high = 28
            elif it % 3 == 0:
                j_low = 14
                j_high = 24
            elif it % 2 == 0:
                j_low = 4
                j_high = 14
            else:
                j_low = 0
                j_high = 10
            
            for j in range(j_low, j_high):
                if (i == 0 or j == 0):
                    patch_1.append(0)
                else:
                    patch_1.append(image[i-1][j-1])
                
                if (j == 0):
                    patch_2.append(0)
                else:
                    patch_2.append(image[i][j-1])
                    
                if (i == 27 or j == 1):
                    patch_3.append(0)
                else:
                    patch_3.append(image[i+1][j-1])
                    
                if (i == 0):
                    patch_4.append(0)
                else:
                    patch_4.append(image[i-1][j])
                    
                patch_5.append(image[i][j])
                
                if (i == 27):
                    patch_6.append(0)
                else:
                    patch_6.append(image[i+1][j])
                    
                if (i == 0 or j == 27):
                    patch_7.append(0)
                else:
                    patch_7.append(image[i-1][j+1])
                    
                if (j == 27):
                    patch_8.append(0)
                else:
                    patch_8.append(image[i][j+1])
                    
                if (i == 27 or j == 27):
                    patch_9.append(0)
                else:
                    patch_9.append(image[i+1][j+1])
                    
        patch_list.append(patch_1)
        patch_list.append(patch_2)
        patch_list.append(patch_3)
        patch_list.append(patch_4)
        patch_list.append(patch_5)
        patch_list.append(patch_6)
        patch_list.append(patch_7)
        patch_list.append(patch_8)
        patch_list.append(patch_9)
        
    dictionary[image_id] = patch_list

In [128]:
training_dict = {}
testing_dict = {}
MNIST_train_subset = MNIST_train[MNIST_train.index.isin(indices)]
for i in range(6000):
    clear_output(wait = True)
    print(i)
    split_image_into_smaller_patches(i, MNIST_train_subset.loc[:, MNIST_train_subset.columns != 'labels'].iloc[i], training_dict)  

5999


In [133]:
for i in range(10000):
    clear_output(wait = True)
    print(i)
    split_image_into_smaller_patches(i, MNIST_test.loc[:, MNIST_test.columns != 'labels'].iloc[i], testing_dict)

9999


In [182]:
hist_names = []
for i in range(2500):
    hist_names.append("Cluster " + str(i+1))

a = np.zeros(shape=(6000,2500))
b = np.zeros(shape=(10000,2500))
training_histogram = pd.DataFrame(a, columns = hist_names)
testing_histogram = pd.DataFrame(b, columns = hist_names)

In [188]:
for i in range(6000):
    clear_output(wait = True)
    print(i)
    entry = [0] * 2500
    for j in range(144):
        init = patch_cluster.predict([training_dict[i][j]])[0]
        entry_level = cluster_center_collections[init].predict([training_dict[i][j]])[0]
        idx = ((init + 1) * (entry_level + 1)) - 1
        entry[idx] += 1
    training_histogram.loc[i] = entry

5999


In [189]:
for i in range(10000):
    clear_output(wait = True)
    print(i)
    entry = [0] * 2500
    for j in range(144):
        init = patch_cluster.predict([testing_dict[i][j]])[0]
        entry_level = cluster_center_collections[init].predict([testing_dict[i][j]])[0]
        idx = ((init + 1) * (entry_level + 1)) - 1
        entry[idx] += 1
    testing_histogram.loc[i] = entry

9999


# Part C

Train a classifier using this histogram of patches
representation. Evaluate this classifier on the test data.

In [204]:
train_labels = MNIST_train_subset["labels"].reset_index(drop = True)
test_labels = MNIST_test["labels"].reset_index(drop = True)
training_histogram = pd.concat([training_histogram, train_labels], axis=1)
testing_histogram = pd.concat([testing_histogram, test_labels], axis = 1)

In [212]:
rf = RandomForestClassifier(n_estimators = 50, random_state = 498)
rf.fit(training_histogram[training_histogram.columns[:-1]], training_histogram["labels"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=498, verbose=0, warm_start=False)

In [215]:
preds = rf.predict(testing_histogram[testing_histogram.columns[:-1]])

In [218]:
cm = confusion_matrix(test_labels, preds)
cm

array([[ 908,    0,    2,    4,    9,    4,   17,    7,   20,    9],
       [   0, 1113,    5,    4,    4,    2,    1,    2,    4,    0],
       [  31,    4,  823,   55,    4,   34,   12,   18,   40,   11],
       [  18,    3,   20,  902,    3,   17,    8,   24,   11,    4],
       [   1,    6,    7,    0,  906,    0,   20,   10,    3,   29],
       [  13,    4,   28,  103,    9,  668,   19,    4,   30,   14],
       [  43,    5,    5,    3,   19,    8,  859,    2,   10,    4],
       [   7,   14,   22,   42,   16,    2,    2,  866,    5,   52],
       [  47,    5,   22,   29,   30,   24,   25,   23,  744,   25],
       [  18,   14,   12,   15,   24,   12,    2,   45,    9,  858]])

In [219]:
accuracy = accuracy_score(test_labels, preds)
accuracy

0.8647

# Part D

For this part we will be changing the patch sample size for training from 6000 to 8000 and the number of cluster centers from 50 to 30

In [221]:
random.seed(498)
patch_training_df_sample = patch_training_df.sample(8000)
indices = patch_training_df_sample.index

In [222]:
patch_cluster = KMeans(n_clusters=30)
patch_cluster_fit = patch_cluster.fit_predict(patch_training_df_sample)
patch_cluster_centers = patch_cluster.cluster_centers_

In [223]:
cluster_data_collections = []
for i in range(30):
    cluster_data_collections.append(pd.DataFrame(columns=col_names))

In [224]:
closest_center = patch_cluster.predict(patch_training_df)

In [225]:
for i in range(60000):
    cluster_data_collections[closest_center[i]] = cluster_data_collections[closest_center[i]].append(patch_training_df.loc[i])

In [226]:
cluster_center_collections = []
for i in range(30):
    internal_cluster = KMeans(n_clusters=30)
    internal_cluster_fit = internal_cluster.fit_predict(cluster_data_collections[i])
    cluster_center_collections.append(internal_cluster)

In [227]:
training_dict = {}
testing_dict = {}
MNIST_train_subset = MNIST_train[MNIST_train.index.isin(indices)]
for i in range(8000):
    clear_output(wait = True)
    print(i)
    split_image_into_smaller_patches(i, MNIST_train_subset.loc[:, MNIST_train_subset.columns != 'labels'].iloc[i], training_dict)  

7999


In [228]:
for i in range(10000):
    clear_output(wait = True)
    print(i)
    split_image_into_smaller_patches(i, MNIST_test.loc[:, MNIST_test.columns != 'labels'].iloc[i], testing_dict)

9999


In [229]:
hist_names = []
for i in range(900):
    hist_names.append("Cluster " + str(i+1))

a = np.zeros(shape=(8000,900))
b = np.zeros(shape=(10000,900))
training_histogram = pd.DataFrame(a, columns = hist_names)
testing_histogram = pd.DataFrame(b, columns = hist_names)

In [230]:
for i in range(8000):
    clear_output(wait = True)
    print(i)
    entry = [0] * 900
    for j in range(144):
        init = patch_cluster.predict([training_dict[i][j]])[0]
        entry_level = cluster_center_collections[init].predict([training_dict[i][j]])[0]
        idx = ((init + 1) * (entry_level + 1)) - 1
        entry[idx] += 1
    training_histogram.loc[i] = entry

7999


In [231]:
for i in range(10000):
    clear_output(wait = True)
    print(i)
    entry = [0] * 900
    for j in range(144):
        init = patch_cluster.predict([testing_dict[i][j]])[0]
        entry_level = cluster_center_collections[init].predict([testing_dict[i][j]])[0]
        idx = ((init + 1) * (entry_level + 1)) - 1
        entry[idx] += 1
    testing_histogram.loc[i] = entry

9999


In [232]:
train_labels = MNIST_train_subset["labels"].reset_index(drop = True)
test_labels = MNIST_test["labels"].reset_index(drop = True)
training_histogram = pd.concat([training_histogram, train_labels], axis=1)
testing_histogram = pd.concat([testing_histogram, test_labels], axis = 1)

In [233]:
rf = RandomForestClassifier(n_estimators = 50, random_state = 498)
rf.fit(training_histogram[training_histogram.columns[:-1]], training_histogram["labels"])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=498, verbose=0, warm_start=False)

In [234]:
preds = rf.predict(testing_histogram[testing_histogram.columns[:-1]])

In [235]:
cm = confusion_matrix(test_labels, preds)
cm

array([[ 935,    0,    5,    1,    5,    3,   20,    2,    8,    1],
       [   0, 1121,    8,    0,    3,    1,    0,    1,    1,    0],
       [  18,    2,  902,   19,   10,   11,   11,   14,   38,    7],
       [   6,    0,   24,  915,    0,   13,    9,   20,   18,    5],
       [   1,    9,    9,    0,  909,    1,   12,    7,    0,   34],
       [  14,    2,   29,   78,    8,  698,   15,   10,   24,   14],
       [  21,    6,    7,    4,   14,   10,  877,    2,   14,    3],
       [   2,   12,   26,   12,   18,    1,    2,  905,    7,   43],
       [  36,    6,   27,   27,   19,   21,   19,   27,  775,   17],
       [  13,   10,    5,   11,   26,    5,    2,   49,    6,  882]])

In [237]:
accuracy = accuracy_score(test_labels, preds)
accuracy

0.8919

In [238]:
error = 1 - accuracy
error

0.10809999999999997

As we see here we performed better with the modified variables. 

# Part E

We can see that in comparison to our ~11% error rate most of the error rates on the website have a 1~3% error rate. Our classifer did beat the linear classifer (1-layer NN) run with no preprocessing which has a 12% test error rate. Looking at the other linear classifiers we can also see that we've performed around the same. We can safely say that with a bit more tuning our clustering-to-classifer setup is as good as a linear NN classifiers from the website.

# Results

Accuracy from Part C was 0.8647

Accuracy from Part D was 0.8919